<a href="https://colab.research.google.com/github/udita2404/CAR_PRICES_PREDICTION/blob/main/CAR_PRICES_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

In [ ]:

DATA_FILENAME = "car data.csv"
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [ ]:
your_name = "Udita Srivastava" # at least 5 characters
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.Year = dataframe.Year * ord(rand_str[1])/100.
    # scale target
    dataframe.Selling_Price = dataframe.Selling_Price * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['Car_Name'], axis=1)
    return dataframe

dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
279,city,2014.0,6.5625,13.60,40126,Petrol,Dealer,Manual,0
293,city,2010.0,3.4125,9.90,38000,Petrol,Dealer,Manual,0
258,city,2015.0,8.8200,13.60,25000,Petrol,Dealer,Manual,0
278,jazz,2016.0,6.3000,8.40,4000,Petrol,Dealer,Manual,0
134,TVS Apache RTR 160,2017.0,0.6825,0.81,11800,Petrol,Individual,Manual,0


In [ ]:
input_cols = ["Year","Present_Price","Kms_Driven","Owner"]
categorical_cols = ["Fuel_Type","Seller_Type","Transmission"]
output_cols = ["Selling_Price"]

In [ ]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[2.0140e+03, 1.3600e+01, 4.0126e+04, 0.0000e+00],
        [2.0100e+03, 9.9000e+00, 3.8000e+04, 0.0000e+00],
        [2.0150e+03, 1.3600e+01, 2.5000e+04, 0.0000e+00],
        ...,
        [2.0160e+03, 1.4000e+00, 3.5000e+04, 0.0000e+00],
        [2.0170e+03, 8.7000e+00, 2.1200e+04, 0.0000e+00],
        [2.0140e+03, 6.7600e+00, 4.0000e+04, 0.0000e+00]]),
 array([[ 6.5625],
        [ 3.4125],
        [ 8.82  ],
        [ 6.3   ],
        [ 0.6825],
        [ 3.0975],
        [ 6.3   ],
        [ 5.1975],
        [20.7375],
        [15.225 ],
        [ 1.5225],
        [ 0.6825],
        [ 4.2   ],
        [ 8.9775],
        [ 2.0475],
        [ 4.2   ],
        [ 3.255 ],
        [ 0.42  ],
        [ 1.2075],
        [ 4.725 ],
        [ 3.6855],
        [24.675 ],
        [ 5.04  ],
        [ 0.3675],
        [10.1325],
        [ 3.99  ],
        [ 1.2075],
        [ 0.2625],
        [10.185 ],
        [ 0.21  ],
        [ 6.5625],
        [ 4.2   ],
        [ 1.1025],
        [ 

In [ ]:
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

dataset = TensorDataset(inputs, targets)
train_ds, val_ds = random_split(dataset, [228, 57])
batch_size = 128

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [ ]:
input_size = len(input_cols)
output_size = len(output_cols)

class CarsModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                  # fill this (hint: use input_size & output_size defined above)

    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out

    def training_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calcuate loss
        loss = F.l1_loss(out, targets)                         # fill this
        return loss

    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)                           # fill this
        return {'val_loss': loss.detach()}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

model = CarsModel()

list(model.parameters())

[Parameter containing:
 tensor([[ 0.0072, -0.0185,  0.1985, -0.4789]], requires_grad=True),
 Parameter containing:
 tensor([-0.1761], requires_grad=True)]

In [ ]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [ ]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

# Check the initial value that val_loss have
result = evaluate(model, val_loader)
print(result)

{'val_loss': 7092.91552734375}


In [ ]:
# Start with the Fitting
epochs = 90
lr = 1e-8
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6582.2451
Epoch [40], val_loss: 6074.2695
Epoch [60], val_loss: 5564.4272
Epoch [80], val_loss: 5054.9341
Epoch [90], val_loss: 4799.8794


In [ ]:
# Prediction Algorithm
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

# Testing the model with some samples
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([2.0140e+03, 1.4000e+01, 6.3000e+04, 0.0000e+00])
Target: tensor([8.6625])
Prediction: tensor([8475.5596])


In [ ]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([2016.0000,    7.6000, 7000.0000,    0.0000])
Target: tensor([6.5625])
Prediction: tensor([947.9310])
